In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
np.random.seed(5)

In [3]:
tmp_speed1 = np.random.randint(0, 3, size=10000)
tmp_speed2 = np.random.randint(3, 25, size=10000)
tmp_speed11 = np.random.randint(3, 25, size=10000)
tmp_speed22 = np.random.randint(0, 3, size=10000)


tilt1 = np.random.randint(200, 300, size=10000)
tilt2 = np.random.randint(0, 200, size=10000)

speed1 = np.concatenate((tmp_speed1, tmp_speed2))
speed2 = np.concatenate((tmp_speed11, tmp_speed22))
tilt = np.concatenate((tilt1, tilt2))

In [4]:
data = []
for i in zip(speed1, speed2, tilt):
    data.append(list(i))

data = np.array(data)

In [12]:
label = []
for i in data:
    if i[0] >= 14 and i[1] <= 3 and i[2] <= 200:
        label.append(1)
    elif i[0] <=3 and i[1] >= 14 and i[2] <= 200:
        label.append(1)
    else:
        label.append(0)
        
label = np.array(label)

In [13]:
datas = pd.DataFrame({
    'speed1':speed1,
    'speed2':speed2,
    'tilt':tilt,
    'label':label
})
datas

,speed1,speed2,tilt,label
0,2,5,223,0
1,1,20,269,0
2,2,20,221,0
3,2,6,250,0
4,0,23,247,0
...,...,...,...,...
19995,24,0,83,1
19996,11,1,135,0
19997,18,1,72,1
19998,7,2,183,0


In [14]:
x_train, x_test, y_train, y_test = train_test_split(data, label, 
                                                      test_size=0.2,
                                                      random_state=83,
                                                      shuffle=True,
                                                      stratify=label)

In [15]:
ss = StandardScaler()
ss.fit(x_train)
x_train_scaled= ss.transform(x_train)
x_test_scaled= ss.transform(x_test)

In [16]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [18]:
lr = LogisticRegression()
lr.fit(x_train_scaled, y_train)

lr_pred=lr.predict(x_test_scaled)
print(accuracy_score(y_test, lr_pred))

0.9985


In [19]:
gs_dt= GridSearchCV(DecisionTreeClassifier(random_state=221), params, n_jobs=-1)
gs_dt.fit(x_train_scaled, y_train)
dt_clf = gs_dt.best_estimator_

dt_clf_pred=dt_clf.predict(x_test_scaled)
print(accuracy_score(y_test, dt_clf_pred))

0.9985


In [20]:
gs_rf= GridSearchCV(RandomForestClassifier(random_state=221), params, n_jobs=-1)
gs_rf.fit(x_train_scaled, y_train)
rf_clf = gs_rf.best_estimator_

rf_clf_pred=rf_clf.predict(x_test_scaled)
print(accuracy_score(y_test, rf_clf_pred))

0.9985


In [21]:
import time

In [22]:
temp = np.array([[15, 8, 223]])
temp_scaled=ss.transform(temp)

In [23]:
start = time.time()
lr_pred=lr.predict(temp_scaled)
end = time.time()

print(lr_pred)
print(f"{end - start:.100f} sec")

[1]
0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 sec


In [24]:
start = time.time()
dt_clf_pred=dt_clf.predict(temp_scaled)
end = time.time()

print(dt_clf_pred)
print(f"{end - start:.100f} sec")

[1]
0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 sec


In [25]:
start = time.time()
rf_clf_pred=rf_clf.predict(temp_scaled)
end = time.time()

print(rf_clf_pred)
print(f"{end - start:.100f} sec")

[0]
0.0059075355529785156250000000000000000000000000000000000000000000000000000000000000000000000000000000 sec


In [27]:
joblib.dump(dt_clf, './dt_clf.pkl')

['./dt_clf.pkl']

In [ ]:
loaded_model = joblib.load('./dt_clf.pkl')